Install Packages

In [ ]:
%pip install llama-index langchain python-dotenv fastembed qdrant_client sentence_transformers llama-index-llms-mistralai langchain_community ragas
%pip install -U bitsandbytes

Import Modules

In [ ]:
import os
import json
import re
from llama_index.core.schema import Document
from langchain.docstore.document import Document as doct
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader

import asyncio
import random
from langchain import HuggingFaceHub
from langchain.chains.summarize.chain import load_summarize_chain

import logging
from dotenv import load_dotenv
from fastembed import SparseTextEmbedding, TextEmbedding
from qdrant_client import QdrantClient, models
from tqdm import tqdm
from typing import List, Union
from sentence_transformers import CrossEncoder

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain import HuggingFaceHub
from langchain.callbacks import get_openai_callback

from datetime import datetime


Load Configuration File

In [3]:
load_dotenv()

cwd = os.path.dirname(os.getcwd())
with open(os.path.join(cwd,"config","global_config.json")) as f:
    gconf = json.load(f)

Qdrant_URL = gconf['qdrant_URL']
Collection_Name = gconf['Collection_Name'][2]
SumCollection_Name = gconf['Collection_Name'][3]

Create Detailed child Documents and their summaries for hierarchical indexing  

In [ ]:
class CustomTransformation:
    def __call__(self, documents):
        transformed_documents = []
        for doc in documents:
            transformed_content = doc.get_content().lower()
            transformed_content = re.sub(r'\s+', ' ', transformed_content)
            transformed_content = re.sub(r'[^\w\s]', '', transformed_content)
            transformed_documents.append(Document(text=transformed_content, metadata=doc.metadata))
        return transformed_documents
    
def Sentence_Splitter_docs_into_nodes(all_documents):
    try:
        splitter = SentenceSplitter(
            chunk_size=int(gconf['chunk_size']),
            chunk_overlap=int(gconf['chunk_overlap'])
        )

        nodes = splitter.get_nodes_from_documents(all_documents)

        return nodes

    except Exception as e:
        print(f"Error splitting documents into nodes: {e}")
        return []

def save_nodes(nodes, output_file):
    try:
        # Create the directory if it does not exist
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        # Convert the TextNode objects to dictionaries
        nodes_dict = [node.dict() for node in nodes]

        with open(output_file, 'w') as file:
            json.dump(nodes_dict, file, indent=4)
        print(f"Saved nodes to {output_file}")
    except Exception as e:
        print(f"Error saving nodes to file: {e}")
async def exponential_backoff(attempt):
    """
    Implements exponential backoff with a jitter.
    
    Args:
        attempt: The current retry attempt number.
        
    Waits for a period of time before retrying the operation.
    The wait time is calculated as (2^attempt) + a random fraction of a second.
    """
    # Calculate the wait time with exponential backoff and jitter
    wait_time = (2 ** attempt) + random.uniform(0, 1)
    print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")

    # Asynchronously sleep for the calculated wait time
    await asyncio.sleep(wait_time)


async def retry_with_exponential_backoff(coroutine, max_retries=1):
    """
    Retries a coroutine using exponential backoff upon encountering a RateLimitError.
    
    Args:
        coroutine: The coroutine to be executed.
        max_retries: The maximum number of retry attempts.
        
    Returns:
        The result of the coroutine if successful.
        
    Raises:
        The last encountered exception if all retry attempts fail.
    """
    for attempt in range(max_retries):
        try:
            # Attempt to execute the coroutine
            return await coroutine
        except Exception as e:
            # If the last attempt also fails, raise the exception
            if attempt == max_retries - 1:
                raise e

            # Wait for an exponential backoff period before retrying
            await exponential_backoff(attempt)

    # If max retries are reached without success, raise an exception
    raise Exception("Max retries reached")

async def summarize_doc(docs,i):
    final_docs = doct(
        page_content=docs.text,
        metadata=docs.metadata
    )
    i = asyncio.create_task(retry_with_exponential_backoff(summary_chain.ainvoke([final_docs])))
    await i
    summary = i.result()
    summary = summary['output_text']
    summary = summary.split("\n\n\nCONCISE SUMMARY:")[1].strip().replace('\n',' ')
    return doct(
        page_content=summary,
        metadata={"source": docs.metadata["file_path"], "page": docs.metadata["page_label"], "summary": True}
    )

if __name__ == '__main__':
    try:
        # Load data from directory
        documents = SimpleDirectoryReader(input_dir=os.path.join(cwd,gconf['input_directory'])).load_data()
        print(f"Loaded {len(documents)} documents {documents}")
        
        if documents:
            # Apply custom transformation
            custom_transform = CustomTransformation()
            documents = custom_transform(documents)
        
            # Split documents into nodes
            nodes = Sentence_Splitter_docs_into_nodes(documents)
        
            print(f"Created {len(nodes)} nodes")

            # Save nodes to a single JSON file
            output_file = os.path.join(cwd,gconf['node_json'])
            save_nodes(nodes, output_file)

            # Create document-level summaries
            os.environ['HUGGINGFACEHUB_API_TOKEN'] = gconf['huggingface_token']
            summary_llm = HuggingFaceHub(repo_id=gconf['hugging_gen_model'],task=gconf['hugging_gen_task'],
                                        model_kwargs={"temperature":float(gconf['hugging_gen_temperature']),
                                                      "max_new_tokens":int(gconf['hugging_gen_max_token'])})
            summary_chain = load_summarize_chain(summary_llm, chain_type="map_reduce")
            
            batch_size = 5  #Adjust this based on your rate limits
            summaries = []
            for i in range(0, len(documents), batch_size):
                batch = documents[i:i+batch_size]
                batch_summaries = await asyncio.gather(*[summarize_doc(doc,i) for doc in batch])
                summaries.extend(batch_summaries)
                await asyncio.sleep(1)  # Short pause between batches
            print("Summaries",summaries)

        else:
            print("No documents to process.")

    except Exception as e:
        print(f"Error processing documents: {e}")

Create Qdrant Collections and Insert Documents with their summaries

In [ ]:
class QdrantIndexing:
    """
    A class for indexing documents using Qdrant vector database.
    """

    def __init__(self) -> None:
        """
        Initialize the QdrantIndexing object.
        """
        self.data_path = os.path.join(cwd,gconf['node_json'])
        self.embedding_model = TextEmbedding(model_name=gconf['fastembed_dense_model'])
        self.sparse_embedding_model = SparseTextEmbedding(model_name=gconf['fastembed_sparse_model'])
        self.qdrant_client = QdrantClient(
                            url=Qdrant_URL)
        self.metadata = []
        self.documents = []
        self.metadata_sum = []
        self.documents_sum = []
        logging.info("QdrantIndexing object initialized.")

    def load_nodes(self, input_file):
        """
        Load nodes from a JSON file and extract metadata and documents.

        Args:
            input_file (str): The path to the JSON file.
        """
        with open(input_file, 'r') as file:
            self.nodes = json.load(file)

        for node in self.nodes:
            self.metadata.append(node['metadata'])
            self.documents.append(node['text'])
        
        for nodesum in summaries:
            self.documents_sum.append(nodesum.page_content)
        
        logging.info(f"Loaded {len(self.nodes)} nodes from JSON file.")

    def client_collection(self):
        """
        Create a collection in Qdrant vector database.
        """
        bothcollections = [Collection_Name,SumCollection_Name]
        for col in bothcollections:
            if not self.qdrant_client.collection_exists(collection_name=f"{col}"): 
                self.qdrant_client.create_collection(
                    collection_name= col,
                    vectors_config={
                        'dense': models.VectorParams(
                            size=384,
                            distance = models.Distance.COSINE,
                        )
                    },
                    sparse_vectors_config={
                        "sparse": models.SparseVectorParams(
                                index=models.SparseIndexParams(
                                    on_disk=False,              
                                ),
                            )
                        }
                )
                logging.info(f"Created collection '{col}' in Qdrant vector database.")

    def create_sparse_vector(self, text):
        """
        Create a sparse vector from the text using BM42 approach.
        """
        # Generate the sparse vector using BM42
        embeddings = list(self.sparse_embedding_model.embed([text]))[0]

        # Check if embeddings has indices and values attributes
        if hasattr(embeddings, 'indices') and hasattr(embeddings, 'values'):
            sparse_vector = models.SparseVector(
                indices=embeddings.indices.tolist(),
                values=embeddings.values.tolist()
            )
            return sparse_vector
        else:
            raise ValueError("The embeddings object does not have 'indices' and 'values' attributes.")


    def documents_insertion(self):
        points = []
        sumpoints =[]
        for i, (doc, metadata) in enumerate(tqdm(zip(self.documents, self.metadata), total=len(self.documents))):
            # Generate both dense and sparse embeddings
            dense_embedding = list(self.embedding_model.embed([doc]))[0]
            sparse_vector = self.create_sparse_vector(doc)

            # Create PointStruct
            point = models.PointStruct(
                id=i,
                vector={
                    'dense': dense_embedding.tolist(),
                    'sparse': sparse_vector,
                },
                payload={
                    'text': doc,
                    **metadata  # Include all metadata
                }
            )
            points.append(point)
        
        # Upsert points
        self.qdrant_client.upsert(
            collection_name=Collection_Name,
            points=points
        )

        logging.info(f"Upserted {len(points)} points with dense and sparse vectors into Qdrant vector database.")

        for j, (docsum, metadata) in enumerate(tqdm(zip(self.documents_sum, self.metadata), total=len(self.documents_sum))):
            # Generate both dense and sparse embeddings
            dense_embeddingsum = list(self.embedding_model.embed([docsum]))[0]
            sparse_vectorsum = self.create_sparse_vector(docsum)

            # Create PointStruct
            pointsum = models.PointStruct(
                id=j,
                vector={
                    'dense': dense_embeddingsum.tolist(),
                    'sparse': sparse_vectorsum,
                },
                payload={
                    'text': docsum,
                    **metadata  # Include all metadata
                }
            )
            sumpoints.append(pointsum)

        # Upsert sumpoints
        self.qdrant_client.upsert(
            collection_name=SumCollection_Name,
            points=sumpoints
        )

        logging.info(f"Upserted {len(points)} points with dense and sparse vectors into Qdrant vector database.")

    
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    indexing = QdrantIndexing()
    indexing.load_nodes(indexing.data_path)
    indexing.client_collection()
    indexing.documents_insertion()

Retriever Class - Hierarchical Indexing Technique with hybrid search

In [4]:
class Hybrid_search():
    """
    class for performing hybrid search using dense and sparse embeddings.
    """

    def __init__(self) -> None:
        """
        Initialize the Hybrid_search object with dense and sparse embedding models and a Qdrant client.
        """
        self.embedding_model = TextEmbedding(model_name=gconf['fastembed_dense_model'])
        self.sparse_embedding_model = SparseTextEmbedding(model_name=gconf['fastembed_sparse_model'])
        self.qdrant_client = QdrantClient(
            url=Qdrant_URL,
            timeout=30
        )

    def metadata_filter(self, file_names: Union[str, List[str]]) -> models.Filter:
        
        if isinstance(file_names, str):
            # Single file name
            file_name_condition = models.FieldCondition(
                key="file_name",
                match=models.MatchValue(value=file_names)
            )
        else:
            # List of file names
            file_name_condition = models.FieldCondition(
                key="file_name",
                match=models.MatchAny(any=file_names)
            )
        
        print("file_name_condition",file_name_condition)

        return models.Filter(
            must=[file_name_condition]
        )

    def query_hybrid_search(self, query, colname, metadata_filter=None, limit=1):
        
        # Embed the query using the dense embedding model
        dense_query = list(self.embedding_model.embed([query]))[0].tolist()

        # Embed the query using the sparse embedding model
        sparse_query = list(self.sparse_embedding_model.embed([query]))[0]

        results = self.qdrant_client.query_points(
            collection_name=colname,
            prefetch=[
                models.Prefetch(
                    query=models.SparseVector(indices=sparse_query.indices.tolist(), values=sparse_query.values.tolist()),
                    using="sparse",
                    limit=limit,
                ),
                models.Prefetch(
                    query=dense_query,
                    using="dense",
                    limit=limit,
                ),
            ],
            query_filter=metadata_filter,
            query=models.FusionQuery(fusion=models.Fusion.RRF),
        )
        # Extract the document number, score, and text from the payload of each scored point
        documents = [{'docum':point.payload['text'],'page':point.payload['page_label']} for point in results.points]

        return documents
    
    def query_hybrid_search_with_filter(self, query, pg_num, colname, metadata_filter=None, limit=1):
        
        # Embed the query using the dense embedding model
        dense_query = list(self.embedding_model.embed([query]))[0].tolist()

        # Embed the query using the sparse embedding model
        sparse_query = list(self.sparse_embedding_model.embed([query]))[0]

        results = self.qdrant_client.query_points(
            collection_name=colname,
            prefetch=[
                models.Prefetch(
                    query=models.SparseVector(indices=sparse_query.indices.tolist(), values=sparse_query.values.tolist()),
                    using="sparse",
                    limit=limit,filter=models.Filter(must=models.FieldCondition(
                    key="page_label",
                    match=models.MatchValue(value=pg_num),
                )
                )),
                models.Prefetch(
                    query=dense_query,
                    using="dense",
                    limit=limit,filter=models.Filter(
                must=models.FieldCondition(
                    key="page_label",
                    match=models.MatchValue(value=pg_num),
                )
                )
                ),
            ],
            query_filter=metadata_filter,
            query=models.FusionQuery(fusion=models.Fusion.RRF),
        )
        # Extract the document number, score, and text from the payload of each scored point
        documents = [point.payload['text'] for point in results.points]

        return documents

ReRanker Technique

In [5]:
class reranking():
    def __init__(self) -> None:
        # Load the CrossEncoder model
        self.model = CrossEncoder(gconf['reranker_model'])

    def rerank_documents(self, query, documents):
        # Compute the similarity scores between the query and each document
        scores = self.model.predict([(query, doc) for doc in documents])

        # Sort the documents based on their similarity scores
        ranked_documents = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

        # Select the top 2 documents
        top_documents = [doc for doc, score in ranked_documents[:2]]

        return top_documents

LLM Prompt Engineering

In [6]:
class prompt_template_generation():
    def __init__(self) -> None:
        self.Collection_Name = Collection_Name
        self.SumCollection_Name = SumCollection_Name
        self.search = Hybrid_search()
        self.reranker = reranking()
        self.prompt_str = """You are an AI assistant specializing in explaining complex topics related to Retrieval-Augmented Generation(RAG). Your task is to provide a clear, concise, and informative explanation based on the following context and query.

        Context:
        {context_str}

        Query: {query_str}

        Please follow these guidelines in your response:
        1. Start with a brief overview of the concept mentioned in the query.
        2. Provide at least one concrete example or use case to illustrate the concept.
        3. If there are any limitations or challenges associated with this concept, briefly mention them.
        4. Conclude with a sentence about the potential future impact or applications of this concept.

        Response: Your explanation should be informative yet accessible, suitable for someone with a basic understanding of RAG. If the query asks for information not present in the context, please state that you don't have enough information to provide a complete answer, and only respond based on the given context.
        Output: Remember output should be only the predicted answer.
        """
        self.prompt_tmpl = PromptTemplate(template=self.prompt_str, input_variables=["context_str","query_str"])

    def prompt_generation(self, query: str, filename: str):
        metadata_filter = self.search.metadata_filter(filename)
        cont_st_time = datetime.now() 
        sumresults = self.search.query_hybrid_search(query, self.SumCollection_Name, metadata_filter)

        relevant_chunks = []
        for summary in sumresults:
            # For each summary, retrieve relevant detailed chunks
            page_number = summary["page"]
            page_chunks  = self.search.query_hybrid_search_with_filter(summary["docum"], page_number, self.Collection_Name, metadata_filter)
            relevant_chunks.extend(page_chunks)
        
        cont_ed_time = datetime.now()
        cont_time = round((cont_ed_time - cont_st_time).total_seconds(),2)

        rank_st_time = datetime.now()
        reranked_documents = self.reranker.rerank_documents(query, relevant_chunks)
        rank_ed_time = datetime.now()
        rank_time = round((rank_ed_time - rank_st_time).total_seconds(),2)
        context = "/n/n".join(reranked_documents)
        
        prompt_templ = self.prompt_tmpl

        return prompt_templ, context, cont_time, rank_time



Define LLM Model

In [7]:
def llm_model():
    os.environ['HUGGINGFACEHUB_API_TOKEN'] = gconf['huggingface_token']
    hugllm = HuggingFaceHub(repo_id=gconf['hugging_gen_model'],task=gconf['hugging_gen_task'],
                                                model_kwargs={"num_beams":3,"top_k":1,"temperature":float(gconf['hugging_gen_temperature']),
                                                            "max_new_tokens":int(gconf['hugging_gen_max_token'])})
    return hugllm

LLM Inference

In [8]:
def inference(inputdata):
    outobj = prompt_template_generation()
    input_pdffiles = os.listdir(os.path.join(cwd,gconf['input_directory']))
    prompt_tmpl, contextpmt, cont_time, rank_time = outobj.prompt_generation(query=inputdata, filename=input_pdffiles)

    hllm = llm_model()
    llm_chain = LLMChain(prompt=prompt_tmpl, 
                        llm=hllm)

    def count_tokens(chain, query, cont):
        with get_openai_callback() as cb:
            gen_st_time = datetime.now()
            resp = chain.run({"context_str":cont,"query_str":query})
            gen_ed_time = datetime.now()
            gentime = round((gen_ed_time - gen_st_time).total_seconds(),2)
            print(resp)
            print(f'Spent a total of {cb.total_tokens} tokens')
            print(f"Prompt Tokens: {cb.prompt_tokens}")
            print(f"Completion Tokens: {cb.completion_tokens}")
            print(f"Total Cost (USD): ${cb.total_cost}")
        
        resp = resp.split('\n        \n')[-1].replace('\n',' ')

        return resp, gentime
    res, gentime = count_tokens(llm_chain, inputdata, contextpmt)

    datarec = {
        "question": [inputdata],
        "answer": [res],
        "contexts": [[contextpmt]]
    }
    return datarec, cont_time, gentime, rank_time

Test retriever and generation for complex queries

In [ ]:
#inputdata = "what is the formula for Property tax liability?"
#inputdata = "what is property tax meaning?"
#inputdata = "Connecticut is not plagued by?"
#inputdata = "What are the components for determining the property tax liabilities?"
#inputdata = "What is the homestead value for real property used for Minnesota Analysis?"
#inputdata = "What are the two cities where assessment limits reduce taxes by 60 percent?"
inputdata = "what is the formula to calculate the Net Tax Bill?"

st_time = datetime.now()
response, cont_time, gentime, ranktime = inference(inputdata)
ed_time = datetime.now()
response_time = round((ed_time - st_time).total_seconds(),2)
print("Generation Response:",response)
print(f"Context_Retrieval_Latency:    {cont_time} secs")
print(f"ReRanking_Latency:            {ranktime} secs")
print(f"RESPONSE_Generation_Latency:  {gentime} secs")
print(f"Overall_RESPONSE_TIME:        {response_time} secs")

%reset_selective -f hugllm
%reset_selective -f hllm
%reset_selective -f resp